# EDA for the RF dataset

In [1]:
import rf_metric

data_dir = rf_metric.utils.config["RF_DATA_DIR"]
print(f"Using data directory: {data_dir}")

Using data directory: /home/nishaq/Documents/research/data/rf-data


In [2]:
import numpy

mod_mode = rf_metric.data.ModulationMode.ASK_4  # type: ignore
noise_level = rf_metric.data.NoiseLevel.POS_20  # type: ignore

dataset = numpy.asarray(mod_mode.read(noise_level=noise_level))  # type: ignore
print(f"Dataset shape for {mod_mode} at {noise_level}: {dataset.shape}")

Dataset shape for ModulationMode.ASK_4 at NoiseLevel.POS_20: (4096, 1024, 2)
